In [1]:
!pip install torchview

In [2]:
import torch
import torchvision
import torchvision.datasets
import torchvision.transforms
import torch.utils.data
import torch.nn
import numpy as np
import os
import time
import graphviz
import torchvision
from torchvision import datasets, models, transforms
from torchview import draw_graph
from matplotlib import pyplot as plot

In [3]:
def denorm (img, a = (0.5, 0.5, 0.5), b = (0.5, 0.5, 0.5)):
  img = img * torch.tensor(a).reshape(3, 1, 1) + torch.tensor(b).reshape(3, 1, 1)
  return img

#  Загрузка данных

In [4]:
dir_name = os.getcwd()
batch_size = 128

In [5]:
#
transforms = models.ResNet50_Weights.DEFAULT.transforms()

train_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = True, download = True,
    transform = transforms
)
test_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = False, download = True,
    transform = transforms
)



100%|██████████| 170498071/170498071 [00:03<00:00, 43186331.82it/s]


Extracting /content/cifar-10-python.tar.gz to /content
Files already downloaded and verified


In [6]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size = batch_size, shuffle = False
)

In [8]:
model = models.resnet50(models.ResNet50_Weights.DEFAULT)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 145MB/s]


In [9]:
for params in model.parameters():
    params.requires_grad = False

In [10]:
in_features = model.fc.in_features

model.fc = torch.nn.Linear(in_features, 10)

# Обучение модели

In [11]:
def get_loss(model, data_loader):
    loss = 0
    n = 0
    with torch.no_grad():
        losses = []
        for images, labels in data_loader:
            labels = labels.to(device)
            images = images.to(device)
            outputs = model(images)
            loss += loss_function(outputs, labels)
            losses.append(loss)
    return epoch_loss(losses)

In [13]:
learning_rate = 0.01
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

def get_accuracy(model, data_loader):
    tp = 0
    n = 0
    with torch.no_grad():
        for images, labels in data_loader:
            labels = labels.to(device)
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n += labels.size(0)
            tp += (predicted == labels).sum()
    return tp / n



def epoch_loss(losses):
    return torch.stack(losses).mean()

start_time = time.time()
for epoch in range(num_epochs):
    start_epoch = time.time()
    losses = []
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()

        losses.append(loss)

        loss.backward()
        optimizer.step()

    total_time_epoch = time.time() - start_epoch
    print('Epoch[{}]: Loss = {:.4f}, Train accuracy = {:.4f}, Time epoch = {:.4f}'.
          format(epoch + 1,
                 epoch_loss(losses),
                 get_accuracy(model, train_loader),
                 total_time_epoch))
total_time = time.time() - start_time
print('Total time {}'.format(total_time))

Epoch[1]: Loss = 0.7413, Train accuracy = 0.7903, Time epoch = 165.9670
Epoch[2]: Loss = 0.5922, Train accuracy = 0.8303, Time epoch = 171.4727
Epoch[3]: Loss = 0.5524, Train accuracy = 0.8079, Time epoch = 173.5022
Epoch[4]: Loss = 0.5337, Train accuracy = 0.8313, Time epoch = 170.3844
Epoch[5]: Loss = 0.5369, Train accuracy = 0.8414, Time epoch = 169.1363
Total time 1713.3508706092834


# Тестирование модели

In [14]:
print('Loss = {:.4f}, Test accuracy = {:.4f}'.format(
    get_loss(model, test_loader),
    get_accuracy(model, test_loader)))

Loss = 59.5664, Test accuracy = 0.7805


In [15]:
model



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

-------------

2

In [14]:
transforms = models.GoogLeNet_Weights.DEFAULT.transforms()

train_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = True, download = True,
    transform = transforms
)
test_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = False, download = True,
    transform = transforms
)


In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size = batch_size, shuffle = False
)

In [ ]:
model = models.googlenet(models.GoogLeNet_Weights.DEFAULT)

In [ ]:
for params in model.parameters():
    params.requires_grad = False

In [ ]:
in_features = model.fc.in_features

model.fc = torch.nn.Linear(in_features, 10)

In [ ]:
learning_rate = 0.01
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

def get_accuracy(model, data_loader):
    tp = 0
    n = 0
    with torch.no_grad():
        for images, labels in data_loader:
            labels = labels.to(device)
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n += labels.size(0)
            tp += (predicted == labels).sum()
    return tp / n



def epoch_loss(losses):
  return torch.stack(losses).mean()

start_time = time.time()
for epoch in range(num_epochs):
    start_epoch = time.time()
    losses = []
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()

        losses.append(loss)

        loss.backward()
        optimizer.step()

    total_time_epoch = time.time() - start_epoch
    print('Epoch[{}]: Loss = {:.4f}, Train accuracy = {:.4f}, Time epoch = {:.4f}'.
          format(epoch + 1,
                 epoch_loss(losses),
                 get_accuracy(model, train_loader),
                 total_time_epoch))
total_time = time.time() - start_time
print('Total time {}'.format(total_time))

In [ ]:
print('Loss = {:.4f}, Test accuracy = {:.4f}'.format(
    get_loss(model, test_loader),
    get_accuracy(model, test_loader)))

In [ ]:
model

---------------------
3

In [ ]:
transforms = models.ConvNeXt_Base_Weights.DEFAULT.transforms()

train_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = True, download = True,
    transform = transforms
)
test_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = False, download = True,
    transform = transforms
)


In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size = batch_size, shuffle = False
)

In [ ]:
model = models.convnext(models.ConvNeXt_Base_Weights.DEFAULT)

In [ ]:
for params in model.parameters():
    params.requires_grad = False

In [ ]:
model.fc = torch.nn.Sequential(
        torch.nn.Linear(in_features, in_features//2),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features//2, 10),
        )

In [ ]:
learning_rate = 0.01
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

def get_accuracy(model, data_loader):
    tp = 0
    n = 0
    with torch.no_grad():
        for images, labels in data_loader:
            labels = labels.to(device)
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n += labels.size(0)
            tp += (predicted == labels).sum()
    return tp / n



def epoch_loss(losses):
  return torch.stack(losses).mean()

start_time = time.time()
for epoch in range(num_epochs):
    start_epoch = time.time()
    losses = []
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()

        losses.append(loss)

        loss.backward()
        optimizer.step()

    total_time_epoch = time.time() - start_epoch
    print('Epoch[{}]: Loss = {:.4f}, Train accuracy = {:.4f}, Time epoch = {:.4f}'.
          format(epoch + 1,
                 epoch_loss(losses),
                 get_accuracy(model, train_loader),
                 total_time_epoch))
total_time = time.time() - start_time
print('Total time {}'.format(total_time))

In [ ]:
print('Loss = {:.4f}, Test accuracy = {:.4f}'.format(
    get_loss(model, test_loader),
    get_accuracy(model, test_loader)))

In [ ]:
model

-------------------------------
4

In [ ]:
transforms = models.Swin_V2_S_Weights.DEFAULT.transforms()

train_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = True, download = True,
    transform = transforms
)
test_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = False, download = True,
    transform = transforms
)


In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size = batch_size, shuffle = False
)

In [ ]:
model = models.swin_v2_s(models.Swin_V2_S_Weights.DEFAULT)

In [ ]:
for params in model.parameters():
    params.requires_grad = False

In [ ]:
model.fc = torch.nn.Sequential(
        torch.nn.Linear(in_features, in_features//2),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features//2, 10),
        )

In [ ]:
learning_rate = 0.01
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

def get_accuracy(model, data_loader):
    tp = 0
    n = 0
    with torch.no_grad():
        for images, labels in data_loader:
            labels = labels.to(device)
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n += labels.size(0)
            tp += (predicted == labels).sum()
    return tp / n



def epoch_loss(losses):
  return torch.stack(losses).mean()

start_time = time.time()
for epoch in range(num_epochs):
    start_epoch = time.time()
    losses = []
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()

        losses.append(loss)

        loss.backward()
        optimizer.step()

    total_time_epoch = time.time() - start_epoch
    print('Epoch[{}]: Loss = {:.4f}, Train accuracy = {:.4f}, Time epoch = {:.4f}'.
          format(epoch + 1,
                 epoch_loss(losses),
                 get_accuracy(model, train_loader),
                 total_time_epoch))
total_time = time.time() - start_time
print('Total time {}'.format(total_time))

In [ ]:
print('Loss = {:.4f}, Test accuracy = {:.4f}'.format(
    get_loss(model, test_loader),
    get_accuracy(model, test_loader)))

In [ ]:
model